In [16]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

# Import helpers
import numpy as np
import random
import os

# Import stabel_baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [17]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        self.state = 38+ + random.randint(-3, 3)
        self.shower_length = 60
        
    def step(self, action):
        # Apply temp adj
        self.state += action-1
        
        # Decrease shower time
        self.shower_length -= 1
        
        # Calculate Reward
        if self.state >= 37 and self.state <=39:
            reward = 1
        else:
            reward = -1
            
        if self.shower_length <=0:
            done = True
        else:
            done = False
            
        info = {}
        
        return self.state, reward, done, info
        
    def render(self):
        # Implement viz
        pass
    def reset(self):
        self.state = np.array([38+random.randint(-3, 3)]).astype(float)
        self.shower_length = 60
        return self.state
    

In [18]:
env = ShowerEnv()

In [19]:
env.observation_space.sample()

array([25.578465], dtype=float32)

## 4. Test Environment

In [20]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-46
Episode:2 Score:-34
Episode:3 Score:-60
Episode:4 Score:-30
Episode:5 Score:-34


In [21]:
log_path = os.path.join('Training', 'Logs')


In [22]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [25]:
model.learn(total_timesteps=40_000)


Logging to Training\Logs\PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -34.4    |
| time/              |          |
|    fps             | 474      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -28.6       |
| time/                   |             |
|    fps                  | 418         |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015537164 |
|    clip_fraction        | 0.0517      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -8.19e-05   |